In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as sts

In [7]:
df = pd.read_csv('https://raw.githubusercontent.com/OHrydko/Machine-Learning-Labs/master/lab1/MP-07-Hrydko.csv', 
                 header=0, names=['x0', 'x1', 'x2', 'x3', 'x4', 'x5', 'y'], sep=';',engine='python')
df.head()

,x0,x1,x2,x3,x4,x5,y
0,2114.97,56.1939,97.7716,3601.37,27,39.8544,17.395
1,1164.74,45.0019,63.9749,2026.36,22,42.8624,2.375
2,2192.61,55.3350,93.4638,3336.69,25,38.2677,12.987
3,1168.13,53.4750,63.4258,2812.04,17,39.8232,5.562
4,2124.26,62.2165,90.9051,2420.26,68,39.9775,14.014


In [16]:
df[df.columns[:-1]] = df[df.columns[:-1]]/df[df.columns[:-1]].max()
train = df.iloc[:200]
# визначення параметрів
system_len = len(train.columns)
alpha = 0.9
# знаходження коефіцієнтів
matrix_x = train[train.columns[:-1]].values

solve = np.dot(np.dot(np.linalg.inv(np.dot(np.transpose(matrix_x), matrix_x)), np.transpose(matrix_x)), train['y'].values)
# Оцінка статистичної значимості коефіцієнтів
# Відхилення
q_e = (train['y'] - (train[train.columns[:-1]] * solve).sum(axis = 1))
# Оцінка дисперсії
s_2 = (q_e**2).sum()/(len(train) - system_len)
# Коефіцієнти зв'язку
K_coef = np.sqrt(np.diag(s_2 * np.linalg.inv(np.dot(np.transpose(matrix_x), matrix_x))))

t_student = round(sts.t.ppf((1 + alpha)/2, len(train) - system_len), 2)
t_calculated = (np.absolute(solve)/K_coef).round(2)
weight_coef = t_calculated > t_student

# Розрахунок коефіцієнта множинної детермінації
R = round(np.sqrt((np.dot(np.dot(np.transpose(solve), np.transpose(matrix_x)), train['y'].values) -
             len(train) * train['y'].mean())/\
    (np.dot(np.transpose(train['y'].values), train['y'].values) - len(train) * train['y'].mean())), 2)
print('Значення розподілу t = {} при параметрах alpha = {} & p = {} при'.format(t_student, alpha, len(train) - system_len))
for i in range(len(solve)):
    print('b{} = {} значущий {} значення t роподілу = {} '
          .format(i, round(solve[i], 2), weight_coef[i], t_calculated[i]))
print('Коефіцієнт детермінації R = {}, R^2 = {},оцінка дисперції s^2 = {}'.format(R, round(R**2, 2), round(s_2, 2)))

Значення розподілу t = 1.65 при параметрах alpha = 0.9 & p = 191 при
b0 = -2.91 значущий True значення t роподілу = 1.86 
b1 = 15.48 значущий True значення t роподілу = 2.07 
b2 = 20.04 значущий True значення t роподілу = 3.1 
b3 = -44.38 значущий True значення t роподілу = 5.94 
b4 = 19.24 значущий True значення t роподілу = 1.69 
b5 = -16.93 значущий True значення t роподілу = 2.24 
b6 = 9.82 значущий True значення t роподілу = 3.93 
b7 = -5.29 значущий False значення t роподілу = 0.71 
Коефіцієнт детермінації R = 0.85, R^2 = 0.72,оцінка дисперції s^2 = 25.62
